## Data

In [4]:
# Recimo da ze imamo slike za training in validation
import os

train_dir_name = "../image_slicing/train/"
validation_dir_name = "../image_slicing/validation/"

train_positive_dir = os.path.join(train_dir_name + "positive")
train_negative_dir = os.path.join(train_dir_name + "negative")

validation_positive_dir = os.path.join(validation_dir_name + "positive")
validation_negative_dir = os.path.join(validation_dir_name + "negative")

In [5]:
# print total length of each dataset
print('total training positive images:', len(os.listdir(train_positive_dir)))
print('total training negative images:', len(os.listdir(train_negative_dir)))
print('total validation positive images:', len(os.listdir(validation_positive_dir)))
print('total validation negative images:', len(os.listdir(validation_negative_dir)))

total training positive images: 7
total training negative images: 8
total validation positive images: 3
total validation negative images: 2


In [9]:
# preprocessing needs to be done:
preprocessing_needed = True
img_shape = (200, 200, 3) #??
from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # All images will be rescaled by 1./255

if preprocessing_needed:
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir_name,
        target_size=(img_shape[0], img_shape[1]), # to be discussed?
        batch_size=120,
        class_mode='binary')
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir_name,
        target_size=(img_shape[0], img_shape[1]), # to be discussed?
        batch_size=19,
        class_mode='binary',
        shuffle=False)

Found 15 images belonging to 2 classes.
Found 5 images belonging to 2 classes.


## Model

In [7]:
# imports
import tensorflow as tf
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [10]:


model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = img_shape),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])
print(model.summary())

/Users/domen/mypython/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 120000)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    15,360,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,360,257 (58.59 MB)

 Trainable params: 15,360,257 (58.59 MB)

 Non-trainable params: 0 (0.00 B)

None


In [11]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

TODO: Model checkpoints!! https://keras.io/api/callbacks/model_checkpoint/ 

In [16]:

# BEFORE:
# history = model.fit(train_generator,
#       steps_per_epoch=8,
#       epochs=15,
#       verbose=1,
#       validation_data = validation_generator,
#       validation_steps=8)

# AFTER:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# Checkpoint directory
checkpoint_dir = "checkpoints/"

# Create directory if it does not exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_path = "checkpoints/model_{epoch:02d}-{val_loss:.2f}.keras"

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,  # Save only the best model
                             monitor='val_loss',   # Monitor the validation loss
                             mode='min',           # Minimize the validation loss
                             verbose=1)            # Print out when a model is being saved

In [15]:
history = model.fit(train_generator,
                    steps_per_epoch=8,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator,
                    validation_steps=8,
                    callbacks=[checkpoint]) 

Epoch 1/15


/Users/domen/mypython/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.4667 - loss: 0.8706

2024-03-25 23:19:14.959950: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/local/Cellar/python@3.9/3.9.19/Frameworks/Python.framework/Versions/3.9/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 108.52462, saving model to checkpoints/model_01-108.52.keras


...

2024-03-25 23:19:21.452944: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 23:19:21.521921: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 254ms/step - accuracy: 0.5333 - loss: 29.5857
Epoch 14: val_loss did not improve from 2.89091
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5333 - loss: 16.6420 - val_accuracy: 0.4000 - val_loss: 24.7747
Epoch 15/15


2024-03-25 23:19:21.793938: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 23:19:21.867103: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 254ms/step - accuracy: 0.5333 - loss: 36.6171
Epoch 15: val_loss did not improve from 2.89091
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5333 - loss: 20.5971 - val_accuracy: 0.4000 - val_loss: 12.7305


2024-03-25 23:19:22.139080: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 23:19:22.206383: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


## Loading the model

In [18]:
# Sample evaluation before exit.
model_path = "checkpoints/model_12-2.89.keras"
loaded_model = tf.keras.models.load_model(model_path)
model.evaluate(validation_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.4000 - loss: 25.4611


[25.461069107055664, 0.4000000059604645]